In [35]:
%load_ext autoreload
%autoreload 2

In [36]:
import os 
funcs = os.listdir('functions')

for func in funcs:
    if func.startswith('__') or func == 'import_all_functions.py':
        continue
    func = func.replace('.py','')
    print(f'{func} imported')
    exec(f'from functions.{func} import *')

import_to_df imported
add_had_remod_flag imported
get_kaggle_data imported
add_years_since imported
add_year_remod_pre1950_flag imported
quality_to_number imported
add_dummies_multiple_columns imported
add_value_dummies_multiple_columns imported
process_data imported
sum_onehots imported
add_zero_flag imported


In [60]:
import pandas as pd 
import pandas_profiling as pdp
import numpy as np
import pprint as pp
import seaborn as sns
from matplotlib import pyplot as plt
from catboost import Pool, CatBoostRegressor

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [39]:
files = get_kaggle_data(
    competition="house-prices-advanced-regression-techniques"
    )

files downloaded for house-prices-advanced-regression-techniques competition
   test.csv
   data_description.txt
   train.csv
   sample_submission.csv
located in /competition_data/


In [40]:
input_data = import_to_df(files)

What files did we get?

In [41]:
input_data.keys()

dict_keys(['test', 'train', 'sample_submission'])

In [42]:
input_train_df = input_data['train']
input_test_df = input_data['test']

In [51]:
train_df = process_data(input_train_df)
target = 'SalePrice'
id_row = 'Id'
train_predictors = [
    c for c in train_df.columns 
    if target not in c
    if id_row not in c
]

In [52]:
test_df = process_data(input_test_df)
test_predictors = [
    c for c in test_df.columns 
    if target not in c
    if id_row not in c
]

Only keep columns in both dfs. Needed in case one hots generate unique columns

In [53]:
cols_to_use = list(set(train_predictors).intersection(test_predictors))

In [54]:
len(cols_to_use), len(train_predictors)

(299, 311)

Removed 1 column

In [72]:
X_train = train_df[cols_to_use]
y_train = train_df[target]

X_test = test_df[cols_to_use]
y_test_labels = test_df[id_row]

In [85]:
# initialize Pool
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test) 

param_dist = {
    'depth': [6,8,10],
    'learning_rate' : [0.01, 0.05, 0.1],
    'iterations'    : [30, 50, 100],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}
#Instantiate RandomSearchCV object
cbr = CatBoostRegressor()

random_grid = RandomizedSearchCV(cbr, param_dist, cv =5)

#Fit the model
random_grid.fit(X_train, y_train)

# Print the tuned parameters and score
print(random_grid.best_params_)
print(random_grid.best_score_)
# specify the training parameters 
# model = CatBoostRegressor(
#     iterations=5, 
#     depth=2, 
#     learning_rate=1, 
#     loss_function='RMSE'
# )
#train the model
# model.fit(train_pool)
# make the prediction using the resulting model
# preds = model.predict(test_pool)

6:	learn: 63331.1111201	total: 725ms	remaining: 255ms
37:	learn: 62968.8217085	total: 743ms	remaining: 234ms
38:	learn: 62642.4241541	total: 760ms	remaining: 214ms
39:	learn: 62319.6938427	total: 776ms	remaining: 194ms
40:	learn: 62033.5395782	total: 792ms	remaining: 174ms
41:	learn: 61723.6266532	total: 809ms	remaining: 154ms
42:	learn: 61401.9725999	total: 825ms	remaining: 134ms
43:	learn: 61118.0338932	total: 841ms	remaining: 115ms
44:	learn: 60820.3036122	total: 857ms	remaining: 95.3ms
45:	learn: 60507.3271044	total: 876ms	remaining: 76.2ms
46:	learn: 60222.8285269	total: 896ms	remaining: 57.2ms
47:	learn: 59876.3071992	total: 916ms	remaining: 38.2ms
48:	learn: 59581.3068975	total: 933ms	remaining: 19ms
49:	learn: 59333.3774788	total: 951ms	remaining: 0us
0:	learn: 80780.0355211	total: 14.8ms	remaining: 725ms
1:	learn: 80272.6468614	total: 28.1ms	remaining: 675ms
2:	learn: 79845.3640258	total: 42.4ms	remaining: 664ms
3:	learn: 79400.8024405	total: 60.7ms	remaining: 698ms
4:	learn: 

Use these params to start grid search

In [87]:
param_grid = {
    'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.3, 0.4],
    'iterations': [60, 80, 100, 150, 200], 
    'depth': [4,7,8,10,12,24],
    'l2_leaf_reg': [5, 7, 8, 9],
    'random_strength': [1,2,3]
}

grid_search = GridSearchCV(estimator = cbr, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_params_)
print(grid_search.best_score_)

# make preds and submit

In [77]:
y_pred = random_grid.predict(X_test)

submission_df = pd.DataFrame()
submission_df[id_row] = y_test_labels
submission_df[target] = y_pred

In [78]:
submission_df.head()

,Id,SalePrice
0,1461,124061.734713
1,1462,156696.494915
2,1463,178224.018442
3,1464,180193.317506
4,1465,191269.212375


In [79]:
input_data['sample_submission'].head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [80]:
submission_df.to_csv('submission1.csv', index=False)

In [83]:
!kaggle competitions submit -f 'submission1.csv' -m 'first attempt random hypers catboost' house-prices-advanced-regression-techniques

100%|██████████████████████████████████████| 33.7k/33.7k [00:02<00:00, 16.7kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques

In [84]:
!kaggle competitions submissions -c house-prices-advanced-regression-techniques

fileName         date                 description                           status    publicScore  privateScore  
---------------  -------------------  ------------------------------------  --------  -----------  ------------  
submission1.csv  2021-05-03 11:30:27  first attempt random hypers catboost  complete  0.15473      None          
